In [2]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [3]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [4]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [5]:
#Total columns 
total_col = len(df.columns)
print(f"total columns = {total_col}")

total columns = 19


In [6]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
#Converting the duration to minutes
df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)

In [7]:
df_SD = df['duration'].std()
print(f"The standard deviation: {df_SD:.2f}")

    

The standard deviation: 42.59


In [20]:
def remove_outliers(df):
    original_count = len(df)
    filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()
    filtered_count = len(filtered_df)
    percentage_left = (filtered_count / original_count) * 100 if original_count > 0 else 0
    print(f"Records left after dropping the outliers: {percentage_left:.2f}%")
    return filtered_df



In [21]:
filtered_df = remove_outliers(df)


Records left after dropping the outliers: 98.12%


In [22]:
print("Unique PU:", filtered_df['PULocationID'].nunique())
print("Unique DO:", filtered_df['DOLocationID'].nunique())

Unique PU: 255
Unique DO: 260


In [12]:
#X = pd.get_dummies(df[['PULocationID', 'DOLocationID']].astype(str))
# X = pd.get_dummies(filtered_df[['PULocationID', 'DOLocationID']].astype(str))

In [27]:
encoder = OneHotEncoder(sparse=True)
X = encoder.fit_transform(filtered_df[['PULocationID', 'DOLocationID']].astype(str))

In [28]:
print(f"The shape of the feature matrix for the data is: {X.shape}")
print(f"the feature columns: {X.shape[1]}")


The shape of the feature matrix for the data is: (3009173, 515)
the feature columns: 515


In [29]:
#fitting regression model 

lr = LinearRegression()

In [33]:
import numpy as np

In [31]:
lr.fit(X, filtered_df['duration'])

y_pred = lr.predict(X)


In [34]:
rmse = np.sqrt(mean_squared_error(filtered_df['duration'], y_pred))
print(f"RMSE: {rmse:.2f}")

RMSE: 7.65


In [38]:
from sklearn.model_selection import train_test_split



In [39]:
X_train, X_val, y_train, y_val = train_test_split(
    X, 
    filtered_df['duration'], 
    test_size=0.2,  
    random_state=42 
)


In [40]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [41]:
y_pred = lr.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"Validation RMSE: {rmse:.2f}")

Validation RMSE: 7.66
